# Session 6 - Semantic Image Segmentation 

------

## <u>강의 내용 정리</u>

### Semantic Image Segmentation 문제 소개  

:: 전체 이미지 pixel 에서 의미있는 부분끼리 묶어서 Prediction하는 기법 (Dense Prediction)

$\rightarrow$ 자율주행 등의 문제를 풀기 위해 선행된다. 

### 전통적 방법론 

* **Mean Shift ( 클러스터링 이용한 방법 )**

  * 과정 순서 

    1. 모든 이미지 픽셀들을 초기값 ($y_0$)로 지정 

    2. 각각의 점을 Kernel Function을 통과시켜 Kernel windows( h ) 안의 모든 점을 해당 위치로 이동 시킨다. 

        :: $y_{t+1} = \dfrac{\sum^n_{i=0} s_ik(\dfrac{x_i - y_t}{h})}{\sum^n_{i =1}k(\dfrac{x_i - y_t}{h})}$  

    3. 이를 임계치 이하가 될 때까지 반복 한다. 

       $||y_{t+1} - y_t|| \leq \varepsilon$ 

  * 직관적인 이해 - Mean shift 알고리즘의 동작과정은 샘플들이 밀집된 방향으로 이동되게 한다. 

  * 장 / 단점

    - 장점 

      1. hyperparameter (클러스터링 개수 )를 지정해 줄 필요가 없다. ( 알고리즘이 자동으로 찾게 된다. )
      2. 설정해야하는 것은 Kernel의 window size ( h )밖에 없다. 

    - 단점 

      1. Kernel window size를 잘 정해줘야한다. 

      2. 고차원 데이터에는 잘 동작하지 않는다.

         ( 오차의 임계치는 어떻게 설정하는지 언급 안 하셨다. )  

  * 성능 - K-means의 결과보다, 많이 향상 되었고, 어느 정도 인식 가능한 수준의 결과물이 나왔다. 

* **Boundary를 이용한 방법 ( Watershed )**

  ( 이부분에 대해 연사님의 설명이 좋았음 ) :: Morphological Watersheds

  1. 사진의 지역 극소점( Reginal minima, 관심 병변 )에 구멍을 낸다. 

  2. 3D topography를 밑바닥 부터 물을 붓듯이  점차적으로 채운다. 

  3. 점차 채워지다가, 두 경계가 합쳐지기 전에 각 경계에 댐을 쌓는다.

    **장 / 단점**

    장점 - 빠름, 경계부분 보존 가능

    단점 - Segmentation Label 이 많아질수록 성능 저하, Noise에 민감하게 반응

* **Graph를 이용한 방법 - Cut을 이용해 그래프 분리**

  1. 모든 이미지 픽셀들을 그래프 상의 Node로 지정한다. 

  2. 각 픽셀들의 유사도를 $w_{ij}$로 나타내어 **Edge**로 지정한다. 

     * $w_{ij}$는 여러가지 형태로 정의 될 수 있다. ( 밝기 값, 화면상의 거리, 텍스쳐간의 유사도 )

       **Intensity** - $aff(x, y) = \exp\{-(\dfrac{1}{2\sigma^2_i})(||I(x) - I(y)||^2)\}$

       **Distance** - $aff(x, y) = \exp\{-(\dfrac{1}{2\sigma^2_d})(||x-y||^2)\}$

       **Texture** - $aff(x, y) = \exp\{-(\dfrac{1}{2\sigma^2_t})(||c(x) - c(y)||^2)\}$

     * Fully connected Graph

       $\rightarrow$ nodes for every pixel $\rightarrow$ link them $\rightarrow$ similarity $w_{ij}$ for each links (링크 간의 거리가 된다. )

     * Normarlized Cuts - break graph(link) into segments

       - 각 Segment를 가로지르는 link들을 삭제 (Low similarity( 먼 거리 ) 는 삭제 됨으로 Cost func(similartiy의 합)은 감소하게 된다. )
       - 결과적으로 비슷한 애들끼리는 링크가 살아 있어 같은 Segment에 포함되고, 그렇지 않은 애들은 연결이 끊겨 다른 Segment에 포함된다. 

     * Minimum Cuts - Cut(A, B)

       Cut(A, B) = 유사도 $w_{ij} (=c_{p, q})$ 의 합으로 표현됨 (거리가 멀면, 유사도는 줄어든다. 유사도의 합을 최대화 하는 것이 목적)

       Cut(A, B)가 작은 에지를 자르게 되면, 서로 분리가 된다. 

       $\rightarrow$ Cut(A, B)가 가장 작은 부분을 끊는다는 의미로 이 알고리즘을 **Minimum Cut**이라고 부른다. 

       * 그러나 Minimum Cut은 작은 그래프를 선호하는 문제점이 있어 상황에 따라 성능이 안좋을 수 있다. 

       * 그래서 **Nomalized Min Cut 알고리즘**은 큰그래프에 가중치를 주어서 정규화한 Min cut이다. 

         :: volume(A) - A와 닿아있는 모든 edge들의 합 -> 그래프가 커질 수록 커진다. 

         $Ncut(A, B) = \dfrac{cut(A, B)}{volume(A)} + \dfrac{cut(A, B)}{volume(B)}$

     * 장 단점 

       장점 : 다양한 상황에서 어느정도 좋은 성능을 보증한다. 

       단점 : 수행속도가 느리다. 몇 개로 그래프를 분할 할지 선택해야한다. 

  * 전통적 방법 VS 딥러닝 

    - 전통적인 방법은 Hand-crafted Feature를 사용하는데 반해, 딥러닝(특히 CNN)은 End2End model로 직접 Feature extraction을 몇단계에 걸쳐서 수행할 수 있다. 

  * Machine Learning의 기본 Process

    1. 학습하고자하는 가설($\theta$)를 수학식으로 표현 
    2. 가설에 관한 모델의 성능을 측정할 수 있는 비용 함수 정의 
    3. 비용함수를 최소화하는 알고리즘 설계 

  * CNN의 기본

    Convolution(합성곱) : $ (f * g)(t) = \int_{-\inf}^{\inf}f(\tau)g(t - \tau)d{\tau}$

    - 하나의 함수와 또다른 함수의 반전 이동한 값을 곱한 다음 구간에 대해 적분하여 새로운 함수를 구하는 연산

    Pooling - Max, Mean 등의 여러 방법이 있다. 구간의 대표값만 남겨 차원을 축소시킨다. 

  * CNN을 Semantic Image Segmentation 어떻게 적용 시킬 것 인가? 

    - 기본적인 것은 Sliding Window 로 이미지를 스캔하면서, 각 픽셀들의 Label을 학습

      -> 계산량이 너무 많고, 해상도가 낮으면, 라벨이 애매한 경우 발생 

    - 앞으로 나올 방법론들이 최근 사용되는 Semantic Image Segmentation방법론이다. 



### Fully Convolutional Networks for Semantic Segmentation( FCN )

  Input     -> Convolutions -> Score -> Prediction

3$\times$H$\times$W          D$\times$H$\times$W         C$\times$H$\times$W      H$\times$W  (every pixel has label)

:: 핵심아이디어 - Classification에 사용된 CNN을 변환해서 Segmentation Task 로 Transfer Learning해서 사용 

* **Classification Network** 

  input -> Convolution -> Fully Connected -> prediction(class)

* **Fully Connected -> 1$\times$1 Convolution** 

  input -> Convolution -> 1$\times$1 Convolution -> prediction(class)

  * Fully Connected 부분을  1x1 Conv 를 함으로써 어떤 사이즈의 image도 다 처리 할 수 있게 됬다. 
    - fully connected의 경우 첫번째 Input(flattened)의 개수가 일정해야하지만, Conv는 그 크기만큼 슬라이딩 해주면 되기 때문에 어떤 Input도 받아들일 수 있게 된다.

  ->원래의 Class 결과를 Heatmap로 나타낼 수 있다. 

* 그래서 이를 **Fully Convolution Network** 라고 한다. 

* 마지막 단에서 크기가 작아진 Feature map을 **Upsampling** 을 통해, 이미지 사이즈 만큼 키운다. 

  * Upsampling - backwards Convolution (deconvolution)

    -> 이 또한 Back Propagation으로 학습시켜, End2End에 모델에 포함 시킬 수 있다. 

    -> 그리고 Non-linear upsampling 이 가능하기 때문에 더욱 좋은 성능을 기대 할 수 있다. 

* **Skip layers**

  Upsampling의 정확도를 높이기 위해 시도 

  * 32x upsampling(FCN-32s)

    마지막 pooling의 결과물의 32x 한 prediction - no skip layer

  * 16x upsampling(FCN-16s)

    마지막 pooling의 결과물x2 * 그 전 Pooing의 결과물 -> 16x 한 prediction

  * 8x upsampling(FCN-8s)

    2전 pooing의 결과물 * 1 전 결과물 x2 * 마지막 결과물 x4 -> 8x한 prediction

* PASCAL VOC 2012 Dataset에서의 성능 비교 

  * Metric

    - pixel accuracy $\dfrac{\sum_i n_{ii}}{\sum_i t_i}$
    - mean accuracy  $(\dfrac{1}{n_{cl}})\sum_i (\dfrac{n_{ii}}{t_i})$
    - mean IU $(1 / n_{cl}) \dfrac{\sum_i n_{ii}}{t_i +\sum_j n{ji}  - n_ii}$
    - frequency weighted IU $(\sum_k t_k)^{-1}\dfrac{\sum_i t_in_{ii}}{t_i + \sum_j n_{ji} - n_{ii}}$

    > 가장 많이 쓰이는 metric 은 IU(Intersection over Union - IoU)라고도 표현함
    >
    > :: IoU = $\dfrac{Area \  of \  overlap}{Area \ of \ Union}$

  * 모든 metric에 대하여 FCN-8s 가 가장 좋은 성능을 냈다. (다른 FCN, SDS, R-CNN 과 비교함 )

    -> 아직 Ground truth (사람)보다는 성능이 좋지 않다. 앞으로 나올 두개의 논문을 FCN을 더욱 개선시킨다. 

### Multi Scale Context Aggregation by Dilated Convolutions

**:: 핵심 아이디어** - Diated Convolution과 FCN 구조개선으로 더욱 좋은 성능의 Semantic Image Segmentation 가능케 했다. 

1 2 3            000100020003000

3 2 1     ->   000300020001000         3x3(예)을 중간 중간 0으로 padding을 주어 15x15로 키운다. 

1 2 3            000100020003000       :: 연산량이 줄어들고, Receptive field를 키운다.(보는 패턴의 범위를 늘림)

* **팽창된(dilated) Convolution** 

  * filter가 커져도 유지해야할 parameter개수는 그대로 유지, 크기가 커진 부분은 모두 0 
  * 장점 - 해상도(Resolution) 손실없이 Receptive Field의 크기를 키울 수 있다. (Pooling size가 동일한데 그 보다 더 세밀한 정보만 pooling하기 때문에, :: Pooling은 CNN 입장에서 한번에 얼마나 넓게 보느냐하는 의미)

* **Front-end Prediction Module** 

  마지막 두번의 Pooling을 제거 

  각각 전의(Pooling layer 이전) 두 번의 Convolution을 2-dilated convolution을 적용

  마지막 Convolution에는 4-convolution을 적용 

  * ResNet(FCN)의 경우

    기존 - 1/2 -> 1/4 -> 1/8 -> 1/16 -> 1/32

    Front end module 적용(FCN) - 1/2 -> 1/ 4 -> 1/8 -> 1/8 -> 1/8

  * PASCAL VOC 2012 Dataset에 대해 성능 향상 

* **Context Module** 

  Multi scale contextual information을 취합함으로써 Dense prediction의 성능을 향상시키기 위해 디자인되었다. 

  각 Convolutions는 Pointwise truncation max(.;0)을 따른다.

  Front end Module의 결과를 Context Module의 Input으로 넣고 실험 

  -> 성능 향상

### RefineNet: Multi-path refinement networks with identity mappings for high-resolution semantic segmentation 

**:: 핵심 아이디어** - Multi Path Fusion을 수행하는 RefineNet구조를 제안, 더욱 좋은 성능의 Semantic Image segmentation결과를 보여준다. (Residual Network with Skip connection)

* Architecture

Multi-Path input -> Adaptive convolution( RCU ; Residual Conv Unit ; Relu- 3x3 conv - Relu - 3x3 conv) -> Multi resolution Fusion(sum of 3x3 conv- Upsampling and 3x3 conv- Upsampling)

-> Chained residual Pooling (sum(sum(5x5 pooling - 3x3 conv), sum(5x5 pooling - 3x3 conv), sum(5x5 pooling - 3x3 conv)))  -> 결과적으로 1/4 까지만 줄어듬

* 다른 모델과의 크기 비교 
  * ResNet - 1/4 -> 1/8 -> 1/16 -> 1/32 -> 1/32
  * Dilated convolution - 1/4 -> 1/8 -> 1/8 -> 1/8 -> 1/8
  * ResNet - 1/4  + 1/8 (x2) + 1/16(x4) + 1/32(x8) -> 1/4
* 성능에 대한 정보는 좀 더 있어야 나올것이다. 